In [0]:
pip install azure-eventhub faker

In [0]:
import json
import time
import uuid
from datetime import datetime, timedelta
from azure.eventhub import EventHubProducerClient, EventData
from faker import Faker
import random

# ----------------------------
# Azure Event Hub Connection
# ----------------------------
# ✅ Replace with your Event Hub details
CONNECTION_STR = dbutils.secrets.get(scope = "telco-secret-keys", key = "telco-order-policy")
EVENTHUB_NAME = dbutils.secrets.get(scope = "telco-secret-keys", key = "telco-order-hub")

# ----------------------------
# Initialize Faker
# ----------------------------
fake = Faker()

# Example plan IDs from your Products/Plans table
PLAN_IDS = list(range(101, 155))  # e.g., 101–200

# ----------------------------
# Function to generate fake order
# ----------------------------
def generate_order(order_id,customer_id):
    return {
        "OrderID": f"O{order_id:05d}",
        "CustomerID": f"{customer_id+1000:04d}",  # Assume 50 customers
        "PlanID": random.choice(PLAN_IDS),
        "OrderDate": (datetime.now() - timedelta(days=random.randint(1, 365))).strftime("%Y-%m-%d"),
        "Status": random.choice(["Completed", "Pending", "Cancelled"]),
        "Amount": random.choice([199, 299, 399, 499, 699, 999, 1299, 1499])
    }

# ----------------------------
# Send Events to Event Hub
# ----------------------------
def send_orders_to_eventhub(num_events=10):
    producer = EventHubProducerClient.from_connection_string(
        conn_str=CONNECTION_STR,
        eventhub_name=EVENTHUB_NAME
    )

    event_batch = producer.create_batch()

    for i in range(num_events):
        order = generate_order(i + 1,i+1)
        event_data = EventData(json.dumps(order))
        event_batch.add(event_data)
        print(f"Sending order: {order}")

    producer.send_batch(event_batch)
    print(f"✅ Sent {num_events} orders to Event Hub")

# ----------------------------
# Run
# ----------------------------
if __name__ == "__main__":
    send_orders_to_eventhub(50)  # Send 20 fake orders
